    mbti, engram, prefer_movie_ids 입력받기
    모델 불러오기
    mbti - movie lens 합친 데이터 불러오기

In [1]:
import pandas as pd

In [ ]:
pd.read_csv('')

In [5]:
import sys
sys.path.append('../interaction_model/')
from inference import Inference

In [6]:
model_path='../interaction_model/similarity'

In [7]:
Inference(model_path, 1, 10)

[780, 648, 736, 32, 733, 608, 480, 2355, 588]